In [ ]:
pip install langchain

In [ ]:
pip install openai

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install faiss-gpu

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

import requests
from langchain.text_splitter import CharacterTextSplitter
import faiss
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import pickle

# from langchain import OpenAI
from langchain.chains import VectorDBQAWithSourcesChain
import argparse

In [ ]:
# Initialize the llm
llm = OpenAI(openai_api_key="ADD_YOUR_API", temperature=0)

In [ ]:
# Function to web scrape and to remove unnecessary parts
def extract_text_from(url):
    """Returns the text content of the given webpage."""
    html = requests.get(url).text
    soup = BeautifulSoup(html, features="html.parser")
    text = soup.get_text()

    lines = (line.strip() for line in text.splitlines())
    return '\n'.join(line for line in lines if line)

In [ ]:
# Sample URL to be passed to the above function
url = "https://lilianweng.github.io/posts/2023-06-23-agent/"

pages = extract_text_from(url)

In [ ]:
# splits based on characters and measure chunk length by number of characters.
text_splitter = CharacterTextSplitter(chunk_size=1500, separator="\n")
docs = text_splitter.split_text(pages)

In [ ]:
docs[2]

'#llm\nIf you want to learn how to create embeddings of your website and how to use a question answering bot to answer questions which are covered by your website, then you are in the right spot.\nThe Github repository which contains all the code of this blog entry can be found here.\nIt was trending on Hacker news on March 22nd and you can check out the disccussion here.\nWe will approach this goal as follows:\nRead more →\nInteractive visualization of stable diffusion image embeddings\nFebruary 26, 2023\n— Written by Marc Päpper\n— ⏰  7 min read\n#machinelearning\n#visualization\nA great site to discover images generated by stable diffusion (or their custom model called aperture) is Lexica.art.\nLexica provides an API which can be used to query images matching some keyword / topic. The API returns image URLs, sizes and other things like the prompt used to generate the image and its seed.\nThe goal of this blog post is to visualize the similarity of images from different categories in

In [ ]:
# Create a vector store of these embeddings
store = FAISS.from_texts(docs, OpenAIEmbeddings())
with open("faiss_store.pkl", "wb") as f:
    pickle.dump(store, f)

In [ ]:
# Asking questions

parser = argparse.ArgumentParser(description='Website Q&A')
parser.add_argument('question', type=str, help= input('Your question for Website'))

# parser.add_argument('question', type=str, help='Your question for Paepper.com')
# promt_user_question=prompt.format(question=user_question)
args = parser.parse_args()

with open("faiss_store.pkl", "rb") as f:
    store = pickle.load(f)

chain = VectorDBQAWithSourcesChain.from_llm(
            llm=OpenAI(temperature=0), vectorstore=store)
result = chain({"question": args.question})

print(f"Answer: {result['answer']}")